In [2]:
import os, json
import csv
import pandas as pd
import nltk
nltk.download('stopwords')  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
stops=stopwords.words('english')
from bs4 import BeautifulSoup
import re
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/weiding/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Preprocessing the Json files

### First define the file path, including the json file, the industry category and the upper category

In [3]:
path_to_json = '/Users/weiding/Documents/Application Project Large Files/vc/sum_all'
path_to_industry = '/Users/weiding/Desktop/industry.dta'
path_to_large_category = '/Users/weiding/Documents/Application Project Large Files/LargeCategory.xlsx'

### The following code reads the Json files, counts the number of files, number of total/valid/empty tabs and prints them out. In addition, it also does the following extracting for the text in the files. 
* Each Json file has the name like "www.zantaz.com_20120414_all.json", we remove the date in the file name and only keep "www.zantaz.com"
* Content of Json file has the key-value format. 
* The key has format like "20170820_About-Us_.html" implying the tab name of this webpage is About-Us. So we only keep the middle part "About-Us" and remove the date and the web suffix.
* Tabs could have subtabs like "3pla_company_jobs". We consider "3pla", "company" and "jobs" as independent subtabs.
* The value of some webpages contains sentences like "The Wayback Machine", which is useless. We ignore such web content and consider its tab as invalid tab.

### It generates two tables. One has two columns: website name and subtab, the other has three columns: website name, tab and sentence of the web content. They will be further processed.

In [4]:
# read all json files from the folder
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
nltk.download('punkt')

# enumerate each file
count_empty_file = 0
company_tab_list = []
company_tab_content_list = []
count_total_tabs = 0
count_empty_tabs = 0
count_valid_tabs = 0
check_file_empty = False
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        if len(json_text) == 0:  # find and ignore empty files
            count_empty_file += 1
        else:
            js = js.split('_20', 1)[0]  # for the file name like "www.zantaz.com_20120414_all.json", 
                                        #only keep the website name or company name and remove the date _2012...

            # processing the tab
            if check_file_empty:
                count_empty_file += 1
            check_file_empty = True
            for key, value in json_text.items(): #tab name is key, web text content is value
                count_total_tabs += 1
                if len(value) == 0:  # remove the empty tab content and tab
                    count_empty_tabs += 1
                elif "The Wayback Machine" in value: #ignore the content containing "The Wayback Machine"
                    continue
                else:
                    check_file_empty = False  # if a tab has useful information, this tab is not empty
                    count_valid_tabs += 1
                    key = key.split('_', 1)[-1]  # remove the date
                    key = key.split('.html', 1)[0]  # remove the .html
                    sub_tabs = key.split('_')
                    for sub_tab in sub_tabs:
                        if len(sub_tab) != 0:
                            company_tab_list.append([js, sub_tab]) #store the subsets in a list
                    sentenceList = nltk.sent_tokenize(value)  # split paragraph as sentences for later removing duplicates
                    for sentence in sentenceList:
                        company_tab_content_list.append([js, key, sentence]) # store the company-tabs-sentences in a list

print("Number of empty file: " + str(count_empty_file))
print("Number of total tabs:" + str(count_total_tabs))
print("Number of empty tabs:" + str(count_empty_tabs))
print("Number of valid tabs:" + str(count_valid_tabs))

[nltk_data] Downloading package punkt to /Users/weiding/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of empty file: 2057
Number of total tabs:235713
Number of empty tabs:21913
Number of valid tabs:213381


### Analyse the tabs and list the unique tabs with their occurences using the table of website and subtabs. The result is stored as CSV file for the later tableau visulization.

In [5]:
company_tab_df = pd.DataFrame(company_tab_list)
company_tab_df.columns = ['Company', 'Tab']
pivoted = pd.pivot_table(company_tab_df, index=['Company','Tab'], aggfunc='size')
company_tab_df_aggregation = pivoted.to_frame().reset_index()
company_tab_df_aggregation.rename(columns={0: 'Occurrences'}, inplace=True)

tab_analyse = company_tab_df_aggregation['Tab'].value_counts()
df_tab_analyse = tab_analyse.to_frame().reset_index()
df_tab_analyse.to_csv("tab_analyse.csv", sep='\t', encoding='utf-8')

### Convert the table of website name, tab and sentence to the Pandas Dataframe, remove the duplicated sentences, then group sentences back to the paragraphs if they are content under the same tab of the same company.

In [6]:
company_tab_content_df = pd.DataFrame(company_tab_content_list)
company_tab_content_df.columns = ['company', 'tab', 'content']
company_tab_content_df.drop_duplicates(subset=['company', 'content'], inplace=True)  # remove duplicate sentences
company_tab_unique_content_df = company_tab_content_df.groupby(['company', 'tab'])['content'].\
    apply(lambda x: '.'.join(x)).reset_index()  # gather sentences together with same company and tab

### Add the category of the company to the table DataFrame 

In [7]:
df_industry = pd.read_stata(path_to_industry)
company_tab_unique_content_df = company_tab_unique_content_df.rename(columns = {'company':'Web'})
df_sum_all_with_label = pd.merge(company_tab_unique_content_df, df_industry, on='Web', how='left') #left join the category
df_sum_all_with_label = df_sum_all_with_label.dropna(subset=['IndustrySegment'])

### Import an xlsx file which projects different IndustrySegments to larger categories

In [8]:
df_upper_category = pd.read_excel(path_to_large_category,header=None)
df_upper_category.columns = ['IndustrySegment', 'Category']
df_upper_category

,IndustrySegment,Category
0,Business Support Services,BUSINESS & FINANCIAL SERVICES
1,Construction and Civil Engineering,BUSINESS & FINANCIAL SERVICES
2,Financial Institutions and Services,BUSINESS & FINANCIAL SERVICES
3,Wholesale Trade and Shipping,BUSINESS & FINANCIAL SERVICES
4,Food and Beverage,CONSUMER GOODS GROUP
5,Household and Office Goods,CONSUMER GOODS GROUP
6,Personal Goods,CONSUMER GOODS GROUP
7,Vehicles and Parts,CONSUMER GOODS GROUP
8,Media and Content,CONSUMER SERVICES GROUP
9,Retailers,CONSUMER SERVICES GROUP


### Combines the table of website name, tab and text content with the category table, so that each company belongs one category. We first ignore the tab information, and combine the paragraphs under different tabs of a same website to a single paragraph.

In [9]:
df_category_analysis = pd.merge(df_sum_all_with_label, df_upper_category, on='IndustrySegment', how='left')
df_category_analysis = df_category_analysis.dropna(subset=['Category'])

df_category_analysis_combined = df_category_analysis.groupby(['Web','Category','IndustrySegment'])['content'].apply(','.join).reset_index()
df_category_analysis_combined.head(5)

,Web,Category,IndustrySegment,content
0,airfox.io,INFORMATION TECHNOLOGY GROUP,Communications and Networking,AirToken is an Ethereum-based ERC-20 digital u...
1,alexotherapeutics.com,HEALTHCARE GROUP,Biopharmaceuticals,"At Alexo Therapeutics, we recognize that the s..."
2,brighterworldenergy.com,ENERGY & UTILITIES GROUP,Utilities,Renewable world uses cookies for tracking..Ple...
3,brown-machine.com,INDUSTRIAL GOODS & MATERIALS GROUP,Machinery and Industrial Goods,BROWN Machine LLC\n 330 N. Ross...
4,corp.kaltura.com,INFORMATION TECHNOLOGY GROUP,Software,This website uses cookies to ensure you get th...


### Clean the website content paragraphs and store it to a csv file.

In [18]:
def content_cleaning(data):
    # remove html tags
    text = BeautifulSoup(data, "lxml").get_text()
    # only get alphabet
    only_letters = re.sub( '[^a-zA-Z]', ' ', text )
    # convert to lower case
    lowercase_words = only_letters.lower().split()
    # remove stopwords
    remove_stopwords = [w for w in lowercase_words if not w in stops]
    return ' '.join(remove_stopwords) 
    
df_category_analysis_combined['clean'] = df_category_analysis_combined['content'].apply(content_cleaning)
df_category_analysis_combined['size']=df_category_analysis_combined['clean'].apply(len)
# filter and remove the companies with alphabets less than 200
df_category_analysis_combined = df_category_analysis_combined[df_category_analysis_combined['size']>200]
df_category_analysis_combined.head(5)

,Web,Category,IndustrySegment,content,clean,size
0,airfox.io,INFORMATION TECHNOLOGY GROUP,Communications and Networking,AirToken is an Ethereum-based ERC-20 digital u...,airtoken ethereum based erc digital utility to...,507
1,alexotherapeutics.com,HEALTHCARE GROUP,Biopharmaceuticals,"At Alexo Therapeutics, we recognize that the s...",alexo therapeutics recognize success company d...,18886
2,brighterworldenergy.com,ENERGY & UTILITIES GROUP,Utilities,Renewable world uses cookies for tracking..Ple...,renewable world uses cookies tracking please r...,4952
3,brown-machine.com,INDUSTRIAL GOODS & MATERIALS GROUP,Machinery and Industrial Goods,BROWN Machine LLC\n 330 N. Ross...,brown machine llc n ross street beaverton mi u...,47277
4,corp.kaltura.com,INFORMATION TECHNOLOGY GROUP,Software,This website uses cookies to ensure you get th...,website uses cookies ensure get best possible ...,114607


### Store the cleaning data for the further usage

In [19]:
df_category_analysis_combined.to_csv("data_after_preprocessing.csv", sep='\t', encoding='utf-8')

### Show number of websites in each category

In [20]:
df_category_analysis_combined['Category'].value_counts()

INFORMATION TECHNOLOGY GROUP          1885
BUSINESS & FINANCIAL SERVICES         1579
CONSUMER SERVICES GROUP               1140
HEALTHCARE GROUP                      1041
INDUSTRIAL GOODS & MATERIALS GROUP     477
CONSUMER GOODS GROUP                   416
ENERGY & UTILITIES GROUP               219
Name: Category, dtype: int64